<a href="https://colab.research.google.com/github/JesseHouse/Turbulence128-on-Phone/blob/main/T128-on-Phone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np, time
N,nu,dt=128,0.005,0.002; M=2*N//3
k=np.fft.rfftfreq(N,1/N); Kz=k[None,None,:]
Kx=np.fft.fftfreq(N,1/N)[:,None,None]; Ky=np.fft.fftfreq(N,1/N)[None,:,None]
Kx=np.broadcast_to(Kx,(N,N,N//2+1)); Ky=np.broadcast_to(Ky,(N,N,N//2+1))
Kz=np.broadcast_to(Kz,(N,N,N//2+1))
mask=(np.abs(Kx)>=M)|(np.abs(Ky)>=M)|(np.abs(Kz)>=M)

x=np.linspace(0,2*np.pi,N,endpoint=False)
X,Y,Z=np.meshgrid(x,x,x,indexing='ij')
u=np.array([np.sin(X)*np.cos(Y)*np.cos(Z),-np.cos(X)*np.sin(Y)*np.cos(Z),np.zeros_like(X)])

def E(u): return 0.5*np.sum(u[0]**2+u[1]**2+u[2]**2)*(2*np.pi/N)**3

print(f"SPECTRALCORE320 — N={N}³ — FINAL 94-LINE PHONE MONSTER")
print(f"Initial energy: {E(u):.10f} | t=0.00s")
np.fft.rfftn(np.random.randn(N,N,N)); t0=time.time(); s=0

while 1:
 s+=1
 uhat=[np.fft.rfftn(u[i]) for i in range(3)]
 uhat=[np.where(mask,0j,h) for h in uhat]
 w0=np.fft.irfftn(1j*(Ky*uhat[2]-Kz*uhat[1]))
 w1=np.fft.irfftn(1j*(Kz*uhat[0]-Kx*uhat[2]))
 w2=np.fft.irfftn(1j*(Kx*uhat[1]-Ky*uhat[0]))
 ux,uy,uz=[np.fft.irfftn(h) for h in uhat]
 nl=[-np.fft.rfftn(w1*uz-w2*uy),-np.fft.rfftn(w2*ux-w0*uz),-np.fft.rfftn(w0*uy-w1*ux)]
 for i in range(3):
  k2=[Kx**2,Ky**2,Kz**2][i]; nl[i]-=nu*k2*uhat[i]; uhat[i]+=dt*nl[i]
 div=Kx*uhat[0]+Ky*uhat[1]+Kz*uhat[2]; K2=Kx**2+Ky**2+Kz**2
 with np.errstate(divide='ignore',invalid='ignore'):
  proj=np.where(K2!=0,div/K2,0)
 uhat[0]-=proj*Kx; uhat[1]-=proj*Ky; uhat[2]-=proj*Kz
 uhat=[np.where(mask,0j,h) for h in uhat]
 u=[np.fft.irfftn(h) for h in uhat]
 if s%100==0:
  elapsed=time.time()-t0
  print(f"Step {s:4d} | E={E(u):.6f} | FPS={s/elapsed:.2f} | t={elapsed:.1f}s")

SPECTRALCORE320 — N=128³ — FINAL 94-LINE PHONE MONSTER
Initial energy: 31.0062766803 | t=0.00s
Step  100 | E=30.944945 | FPS=1.15 | t=86.8s
Step  200 | E=30.882821 | FPS=1.17 | t=170.6s


KeyboardInterrupt: 